In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/Users/iovcharenko/notwork/proj/retail-hack


In [71]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from tqdm.cli import tqdm
from pathlib import Path
from itertools import combinations

In [10]:
data_folder = Path("data")
cheques_df = pd.read_csv(data_folder / "cheques_public.csv", sep=";")
darkstore_df = pd.read_csv(data_folder / "darkstore_map.csv", sep=";")

In [59]:
g = nx.Graph()

for cheque_id, cheque_df in tqdm(cheques_df.groupby("CHEQUEID", sort=True)):
    for _, cheque_item in cheque_df.iterrows():
        # print(cheque_item.LAGERID, cheque_item.KOLVO)
        if g.has_node(cheque_item.LAGERID):
            g.nodes[cheque_item.LAGERID]["num_of_cheques"] += 1
            g.nodes[cheque_item.LAGERID]["overall_amount"] += cheque_item.KOLVO
            
        else:
            node_info = { 
                "num_of_cheques": 1,
                "overall_amount": cheque_item.KOLVO
            }
            g.add_node(cheque_item.LAGERID, **node_info)

    produc_ids =  cheque_df.LAGERID.unique()
    for prod_1, prod_2 in list(combinations(produc_ids, 2)):
        if g.has_edge(prod_1, prod_2):
            g.edges[prod_1, prod_2]['weight'] += 1
        else:
            g.add_edge(prod_1, prod_2, weight=1)


100%|█████████████████████████████████████████████| 5822/5822 [00:01<00:00, 5339.02it/s]


In [117]:
most_popular_prod = max(list(g.nodes.data()), key=lambda n: n[1]["overall_amount"])
most_popular_prod

(44.0,
 {'num_of_cheques': 354,
  'overall_amount': 2254.0,
  'mean_amount': 6.367231638418079})

In [113]:
for n in g.nodes:
    g.nodes[n]["mean_amount"]  = g.nodes[n]["overall_amount"] / g.nodes[n]["num_of_cheques"]

In [118]:
# g.nodes.data()

In [119]:
# list(g.nodes.data())[0]

In [149]:

closest_prods = sorted([(n, g.edges[44, n]) for n in list(g.neighbors(44))], key=lambda n: n[1]["weight"], reverse=True)
closest_prods[:10]

[(15, {'weight': 125}),
 (29, {'weight': 72}),
 (30, {'weight': 48}),
 (7, {'weight': 41}),
 (3, {'weight': 37}),
 (69, {'weight': 36}),
 (116, {'weight': 31}),
 (114, {'weight': 30}),
 (53, {'weight': 28}),
 (42, {'weight': 28})]

In [142]:
# [( g.edges[44, n]) for n in list(g.neighbors(44))]

In [95]:
# g.add_edge(1, 2)
# g.add_edge(1, 3)
# g.number_of_edges()
# g.nodes[1]

In [93]:
# plt.figure(figsize=(25, 25))
# nx.draw(g, width=0.05, with_labels=True)

In [94]:
# cheques_df.LAGERID.unique()